In [ ]:
# Install only what's needed
!pip install torch-geometric -q
!pip install rdkit
import torch
from torch.utils.data import IterableDataset
from torch_geometric.datasets import MoleculeNet
import pandas as pd
import itertools
import sys
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/Project_HIV'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))


Mounted at /content/gdrive
['.git', 'HIV.ipynb', 'README.md', 'code', 'datasets', 'graph_pe_demo.py']


In [ ]:
# Add code directory to path
from torch_geometric.datasets import MoleculeNet
# sys.path.insert(0, os.path.join(os.getcwd(), 'code'))

# # Import the Colab-friendly version
# from data.dataset_colab import PartialHIVDataset

class PartialHIVDataset(IterableDataset):
    """
    IterableDataset that loads only a small portion of the HIV dataset.
    """

    def __init__(self, root='/tmp/HIV', max_samples=10):
        """
        Args:
            root: Root directory for dataset storage
            max_samples: Maximum number of samples to load
        """
        self.root = root
        self.max_samples = max_samples
        self._dataset = None

    def _lazy_load_dataset(self):
        """Lazy load the dataset only when iteration begins"""
        if self._dataset is None:
            print(f"Initializing HIV dataset (will only load {self.max_samples} samples)...")
            # Import here to ensure it's available

            self._dataset = MoleculeNet(root=self.root, name='HIV')
            print(f"Dataset ready! Total size: {len(self._dataset)} molecules")
            print(f"But we'll only load {self.max_samples} of them.\n")

    def parse_molecules(self):
        """
        Parse molecules from the dataset, stopping after max_samples.
        """
        self._lazy_load_dataset()

        for i in range(min(self.max_samples, len(self._dataset))):
            data = self._dataset[i]

            # Extract molecule information
            mol_info = {
                'source': f"molecule_{i}",  # Similar to your CustomIterableDataset
                'target': data.y.item(),    # HIV activity label
                'num_atoms': data.num_nodes,
                'num_bonds': data.num_edges // 2,  # Undirected edges
                'smiles': getattr(data, 'smiles', 'N/A')
            }

            yield mol_info

    def __iter__(self):
        """Iterator with worker support"""
        iterator = self.parse_molecules()
        worker_info = torch.utils.data.get_worker_info()

        if worker_info is not None:
            worker_total_num = worker_info.num_workers
            worker_id = worker_info.id
            return itertools.islice(iterator, worker_id, None, worker_total_num)

        return iterator


# Your code
batch_list = []
batch_size = 40

# Create dataset
dataset = PartialHIVDataset(max_samples=batch_size)
iterator = iter(dataset)
full_batch_data = list(dataset)

df = pd.DataFrame(full_batch_data)

Initializing HIV dataset (will only load 40 samples)...
Dataset ready! Total size: 41120 molecules
But we'll only load 40 of them.



In [ ]:
import codecs

import numpy as np
import torch.nn as nn
from transformers import AutoTokenizer

smiles_list = []
batch_size = 40


# Iterate through the dataset and fill it with the SMILES strings
for _ in range(batch_size):
  mol_info = next(iterator)
  smiles_list.append(mol_info['smiles'])



# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

# First, tokenize without padding to find the actual max length
tokenized_lengths = []
for smiles in smiles_list:
    tokens = tokenizer(smiles, padding=False, truncation=False)
    tokenized_lengths.append(len(tokens['input_ids']))

# Determine max_length from your data
max_length = max(tokenized_lengths)
print(f"Maximum sequence length in data: {max_length}")


# Now tokenize with the data-driven max_length
encoded = tokenizer(
    smiles_list,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

# Create embedding layer
embedding = nn.Embedding(
    num_embeddings=len(tokenizer),
    embedding_dim=512,
    padding_idx=tokenizer.pad_token_id
)

# Get embeddings directly
embeddings = embedding(encoded['input_ids'])
print(f"Embeddings shape: {embeddings.shape}")  # (batch_size, max_length, 512)



Maximum sequence length in data: 61
Embeddings shape: torch.Size([40, 61, 512])


In [ ]:
from scipy.sparse.linalg import eigsh
from rdkit import Chem
def smiles_to_adjacency_matrix(smiles):
    """Convert SMILES to adjacency matrix."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    n_atoms = mol.GetNumAtoms()
    adj_matrix = np.zeros((n_atoms, n_atoms))

    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        # Undirected graph
        adj_matrix[i, j] = 1
        adj_matrix[j, i] = 1

    return adj_matrix

def compute_graph_positional_encoding(adj_matrix, k=30):
    """
    Compute graph positional encodings from eigenvectors of the
    symmetrically normalized graph Laplacian.
    """
    n = adj_matrix.shape[0]

    # Compute degree matrix
    degree = np.sum(adj_matrix, axis=1)
    degree[degree == 0] = 1

    # D^(-1/2)
    d_inv_sqrt = np.diag(1.0 / np.sqrt(degree))

    # Symmetrically normalized Laplacian
    identity = np.eye(n)
    normalized_adj = d_inv_sqrt @ adj_matrix @ d_inv_sqrt
    laplacian = identity - normalized_adj

    # Compute eigenvectors
    if n < k:
        # If graph has fewer nodes than k, pad with zeros
        eigenvalues, eigenvectors = np.linalg.eigh(laplacian)
        # Pad eigenvectors to have k columns
        padded_eigenvectors = np.zeros((n, k))
        padded_eigenvectors[:, :n] = eigenvectors
        return padded_eigenvectors
    else:
        eigenvalues, eigenvectors = eigsh(laplacian, k=k, which='SM')
        return eigenvectors

In [ ]:
# Process all SMILES and compute their graph PEs
graph_pes_list = []
max_nodes = 0

for smiles in smiles_list:
    adj_matrix = smiles_to_adjacency_matrix(smiles)
    if adj_matrix is not None:
        pe = compute_graph_positional_encoding(adj_matrix, k=30)
        graph_pes_list.append(pe)
        max_nodes = max(max_nodes, pe.shape[0])
    else:
        graph_pes_list.append(None)

print(f"Maximum number of atoms in dataset: {max_nodes}")


Maximum number of atoms in dataset: 42


In [ ]:
import torch
import numpy as np
from rdkit import Chem
import re

def generate_random_orthonormal_pe(n_vectors, dim=30):
    """Generate n_vectors random orthonormal vectors of dimension dim."""
    if n_vectors == 0:
        return np.zeros((0, dim))

    # Generate random matrix
    random_matrix = np.random.randn(dim, n_vectors)

    # Use QR decomposition to get orthonormal vectors
    Q, _ = np.linalg.qr(random_matrix)

    # Return first n_vectors columns (transposed to have shape (n_vectors, dim))
    return Q[:, :n_vectors].T

def parse_token_components(token):
    """
    Parse a token to identify atoms and characters.
    Returns: (atom_indices, n_characters)
    """
    # Common atom patterns in SMILES
    atom_pattern = r'(Cl|Br|Si|Mg|Ca|Fe|Al|Na|Li|[BCNOFPSKHIV])'

    # Find all atoms in the token
    atoms = re.findall(atom_pattern, token)

    # Count non-atom characters
    # Remove atoms from token to count remaining characters
    remaining = token
    for atom in atoms:
        remaining = remaining.replace(atom, '', 1)

    # Count actual characters (digits, +, -, =, #, etc.)
    n_characters = len([c for c in remaining])

    return atoms, n_characters

def align_graph_pe_to_tokens(smiles, graph_pe, tokenizer, max_length,random_seed=None):
    """
    Align graph PE to tokens, handling pure characters, atoms, and mixed tokens.

    Args:
        smiles: SMILES string
        graph_pe: Graph positional encoding for atoms (n_atoms, embedding_dim)
        tokenizer: Tokenizer object
        max_length: Maximum sequence length
        random_seed: Random seed for reproducible random PEs
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    # Determine embedding dimension
    if graph_pe is not None:
        embedding_dim = graph_pe.shape[1]
    elif embedding_dim is None:
        embedding_dim = 30  # Default fallback

    mol = Chem.MolFromSmiles(smiles)
    if mol is None or graph_pe is None:
        return torch.zeros(max_length, embedding_dim)

    # Get tokens
    encoding = tokenizer(smiles, padding='max_length', max_length=max_length)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])


    # Initialize token PE
    token_pe = torch.zeros(max_length, embedding_dim)

    # Build atom mapping from SMILES
    atom_symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
    atom_count = {symbol: 0 for symbol in set(atom_symbols)}
    atom_to_idx = {}

    for idx, symbol in enumerate(atom_symbols):
        atom_to_idx[(symbol, atom_count[symbol])] = idx
        atom_count[symbol] += 1

    # Reset atom count for tracking
    current_atom_count = {symbol: 0 for symbol in atom_count}

    # Process each token
    for i, token in enumerate(tokens):
        if token in ['<s>', '</s>', '<pad>']:
            continue

        # Parse token components
        atoms_in_token, n_characters = parse_token_components(token)

        if len(atoms_in_token) == 0 and n_characters > 0:
            # Pure character token - use random orthonormal PE
            random_pes = generate_random_orthonormal_pe(n_characters, embedding_dim)
            token_pe[i] = torch.tensor(random_pes.sum(axis=0))

        elif len(atoms_in_token) > 0 and n_characters == 0:
            # Pure atom token(s) - use graph PE
            atom_pes = []
            for atom_symbol in atoms_in_token:
                if atom_symbol in current_atom_count:
                    atom_key = (atom_symbol, current_atom_count[atom_symbol])
                    if atom_key in atom_to_idx:
                        atom_idx = atom_to_idx[atom_key]
                        atom_pes.append(graph_pe[atom_idx])
                        current_atom_count[atom_symbol] += 1

            if atom_pes:
                # Average the PEs of all atoms in this token
                token_pe[i] = torch.tensor(np.sum(atom_pes, axis=0))

        elif len(atoms_in_token) > 0 and n_characters > 0:
            # Mixed token - combine atom PE and character PE
            # Get atom PEs
            atom_pes = []
            for atom_symbol in atoms_in_token:
                if atom_symbol in current_atom_count:
                    atom_key = (atom_symbol, current_atom_count[atom_symbol])
                    if atom_key in atom_to_idx:
                        atom_idx = atom_to_idx[atom_key]
                        atom_pes.append(graph_pe[atom_idx])
                        current_atom_count[atom_symbol] += 1

            # Get character PEs
            random_pes = generate_random_orthonormal_pe(n_characters, embedding_dim)

            # Combine: sum of atoms + sum of characters
            combined_pe = np.zeros(embedding_dim)
            if atom_pes:
                combined_pe += np.sum(atom_pes, axis=0)
            combined_pe += random_pes.sum(axis=0)

            token_pe[i] = torch.tensor(combined_pe)

    return token_pe



In [ ]:
class EmbeddingWithGraphPE(nn.Module):
    def __init__(self, embed_dim=512, pe_dim=30):
        super().__init__()
        # One-layer projection with GeLU from graph PE to embedding dimension
        # Using standard Laplacian, so pe_dim is just k
        self.pe_projection = nn.Sequential(
            nn.Linear(pe_dim, embed_dim),
            nn.GELU()
        )

    def forward(self, embeddings, graph_pes):
        # token_pes shape: [batch_size, seq_len, pe_dim]
        # embeddings shape: [batch_size, seq_len, embed_dim]

        # Project token PEs to embedding dimension with GeLU
        projected_pes = self.pe_projection(graph_pes)

        # Add to token embeddings
        enhanced_embeddings = embeddings + projected_pes

        return enhanced_embeddings

# Usage
model = EmbeddingWithGraphPE(embed_dim=512, pe_dim=30)
# graph_pes_batch shape: [40, 61, 30] (30 eigenvectors from standard Laplacian)
# embeddings shape: [40, 61, 512]
enhanced_embeddings = model(embeddings, graph_pes_batch)
# Output shape: [40, 61, 512]

In [61]:
import json
import pandas as pd
from collections import Counter
import random

def check_class_balance(data, name="Dataset"):
    """
    Check the balance of classes (0's and 1's) in the dataset.

    Args:
        data: List of dictionaries or DataFrame
        name: Name of the dataset for display

    Returns:
        Dictionary with class counts and percentages
    """
    if isinstance(data, pd.DataFrame):
        targets = data['target'].values
    else:
        # For list of dictionaries in LitGPT format
        targets = [int(item['output']) for item in data]

    class_counts = Counter(targets)
    total = len(targets)

    print(f"\n{name} Class Balance:")
    print(f"Total samples: {total}")
    for class_label in sorted(class_counts.keys()):
        count = class_counts[class_label]
        percentage = (count / total) * 100
        print(f"Class {class_label}: {count} samples ({percentage:.2f}%)")

    return {
        'total': total,
        'class_0': class_counts[0],
        'class_1': class_counts[1],
        'ratio_0': class_counts[0] / total,
        'ratio_1': class_counts[1] / total
    }


def stratified_train_val_split(data, train_ratio=0.8, random_seed=42):
    """
    Create a stratified train/validation split that maintains class balance.

    Args:
        data: List of dictionaries in LitGPT format
        train_ratio: Ratio of training data
        random_seed: Random seed for reproducibility

    Returns:
        train_data, val_data
    """
    random.seed(random_seed)

    # Separate data by class
    class_0_data = [item for item in data if item['output'] == '0']
    class_1_data = [item for item in data if item['output'] == '1']

    # Shuffle each class separately
    random.shuffle(class_0_data)
    random.shuffle(class_1_data)

    # Calculate split indices for each class
    split_idx_0 = int(len(class_0_data) * train_ratio)
    split_idx_1 = int(len(class_1_data) * train_ratio)

    # Split each class
    train_class_0 = class_0_data[:split_idx_0]
    val_class_0 = class_0_data[split_idx_0:]

    train_class_1 = class_1_data[:split_idx_1]
    val_class_1 = class_1_data[split_idx_1:]

    # Combine and shuffle
    train_data = train_class_0 + train_class_1
    val_data = val_class_0 + val_class_1

    random.shuffle(train_data)
    random.shuffle(val_data)

    return train_data, val_data


def convert_to_litgpt_format(df, task_type="classification"):
    """
    Convert the HIV dataset to LitGPT JSONL format with instruction/input/output fields.

    Args:
        df: DataFrame containing the HIV dataset
        task_type: Type of task - "classification", "property_prediction", or "generation"

    Returns:
        List of dictionaries in LitGPT format
    """
    litgpt_data = []

    for idx, row in df.iterrows():
        # Create different instruction templates based on task type
        if task_type == "classification":
            # Binary classification for HIV activity
            instruction = "Classify the following molecule based on its HIV activity. Respond with '1' if the molecule shows HIV activity, or '0' if it does not."
            input_text = f"SMILES: {row['smiles']}"
            output_text = str(int(row['target']))

        else:
            raise ValueError(f"Unknown task_type: {task_type}")

        # Create the LitGPT format entry
        litgpt_entry = {
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        }

        litgpt_data.append(litgpt_entry)

    return litgpt_data


def save_to_jsonl(data, filename):
    """
    Save data to JSONL format (one JSON object per line).

    Args:
        data: List of dictionaries
        filename: Output filename
    """
    with open(filename, 'w') as f:
        for item in data:
            json_line = json.dumps(item)
            f.write(json_line + '\n')
    print(f"Saved {len(data)} entries to {filename}")


# Main execution
if __name__ == "__main__":
    # Note: You'll need to import or define PartialHIVDataset and import pandas as pd
    # from your_module import PartialHIVDataset
    # import pandas as pd

    # Create dataset
    batch_size = 1000  # Adjust based on your needs
    dataset = PartialHIVDataset(max_samples=batch_size)

    # Convert to DataFrame
    full_batch_data = list(dataset)
    df = pd.DataFrame(full_batch_data)

    print(f"Loaded {len(df)} molecules")
    print("\nSample data:")
    print(df.head())

    # Check balance in the original DataFrame
    original_balance = check_class_balance(df, "Original Dataset")

    # Convert to LitGPT format for different task types
    classification_data = convert_to_litgpt_format(df, task_type="classification")

    # Check balance in the converted data
    converted_balance = check_class_balance(classification_data, "Converted Dataset")

    # Create stratified train/validation split
    train_data, val_data = stratified_train_val_split(classification_data, train_ratio=0.8)

    # Check balance in train and validation sets
    train_balance = check_class_balance(train_data, "Training Set")
    val_balance = check_class_balance(val_data, "Validation Set")

    # Compare ratios
    print("\n" + "="*50)
    print("Class Distribution Comparison:")
    print("="*50)
    print(f"Original - Class 0: {original_balance['ratio_0']:.4f}, Class 1: {original_balance['ratio_1']:.4f}")
    print(f"Train    - Class 0: {train_balance['ratio_0']:.4f}, Class 1: {train_balance['ratio_1']:.4f}")
    print(f"Val      - Class 0: {val_balance['ratio_0']:.4f}, Class 1: {val_balance['ratio_1']:.4f}")

    # Calculate difference from original distribution
    train_diff_0 = abs(train_balance['ratio_0'] - original_balance['ratio_0'])
    train_diff_1 = abs(train_balance['ratio_1'] - original_balance['ratio_1'])
    val_diff_0 = abs(val_balance['ratio_0'] - original_balance['ratio_0'])
    val_diff_1 = abs(val_balance['ratio_1'] - original_balance['ratio_1'])

    print(f"\nDifference from original distribution:")
    print(f"Train - Class 0: {train_diff_0:.4f}, Class 1: {train_diff_1:.4f}")
    print(f"Val   - Class 0: {val_diff_0:.4f}, Class 1: {val_diff_1:.4f}")

    # Save to JSONL files
    save_to_jsonl(train_data, "hiv_train.jsonl")
    save_to_jsonl(val_data, "hiv_val.jsonl")

    # Print sample entries to verify format
    print("\nSample LitGPT format entries:")
    for i, entry in enumerate(classification_data[:3]):
        print(f"\nEntry {i+1}:")
        print(f"Instruction: {entry['instruction']}")
        print(f"Input: {entry['input']}")
        print(f"Output: {entry['output']}")

    # Verify JSONL format by reading back
    print("\nVerifying JSONL format...")
    with open("hiv_train.jsonl", 'r') as f:
        first_line = f.readline()
        loaded_entry = json.loads(first_line)
        print("First entry from JSONL file:")
        print(json.dumps(loaded_entry, indent=2))


Initializing HIV dataset (will only load 1000 samples)...
Dataset ready! Total size: 41120 molecules
But we'll only load 1000 of them.

Loaded 1000 molecules

Sample data:
       source  target  num_atoms  num_bonds  \
0  molecule_0     0.0         19         20   
1  molecule_1     0.0         39         44   
2  molecule_2     0.0         21         24   
3  molecule_3     0.0         24         25   
4  molecule_4     0.0         10          9   

                                              smiles  
0  CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...  
1  C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...  
2                   CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21  
3    Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1  
4                             O=S(=O)(O)CCS(=O)(=O)O  

Original Dataset Class Balance:
Total samples: 1000
Class 0.0: 971 samples (97.10%)
Class 1.0: 29 samples (2.90%)

Converted Dataset Class Balance:
Total samples: 1000
Class 0: 971 samples (97.10%)
Class 1: 29 samp